In [ ]:
import numpy as np
from skimage import io

import os
import napari
import pickle

from sklearn.preprocessing import normalize

import morphotrack.flow

In [ ]:
io_directory = '/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/morphotrack'

In [ ]:
# get binary data for blood vesssel
vessel = io.imread(os.path.join(io_directory,'vessel_labkit.tif'))==1

In [ ]:
# make guide vector from the binary image.
# load a mask of a white matter 
white_matter = io.imread(os.path.join(io_directory,'white_matter.tif'))!=0
# load a mask of a layer 1
layer1 = io.imread(os.path.join(io_directory,'l1.tif'))!=0

# make guide vector 
guide_vector = morphotrack.flow.guide_vector_generator_from_binaries(layer1, white_matter)

# or manually make a guide vector
# guide_coordinate1 = np.array([155.,122.,197.])
# guide_coordinate2 = np.array([110.,315.,213.])
# guide_vector = morphotrack.flow.guide_vector_generator(guide_coordinate1, guide_coordinate2)

In [ ]:
vectors, img = morphotrack.flow.get_vectors_from_vessel(vessel, guide_vector, threshold=10, k=27, return_image=True)

In [ ]:
# visualize if needed
viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [ ]:
# Extract the vectors and positions using vessel.
vessel_position = np.array(np.where(vessel)).T

# The first smoothing
smoothed_vectors = morphotrack.flow.smooth_vectors(vessel_position,vectors)

In [ ]:
# visualize if needed
viewer = napari.Viewer()

img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [ ]:
# remove sub-branches for the better fitting
keep = morphotrack.flow.remove_dissimilar_vectors(smoothed_vectors, vectors, threshold='otsu') # dots>=thresh

In [ ]:
# visualize if needed
viewer = napari.Viewer()

img = np.zeros(vessel.shape)
img[tuple(vessel_position[keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='green', blending='additive')
img = np.zeros(vessel.shape)
img[tuple(vessel_position[~keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='magenta', blending='additive')

In [ ]:
# repeat the process to sophisticate the vector smoothing
vessel_position = np.array(np.where(vessel)).T[keep]

# The second smoothing
smoothed_vectors = morphotrack.flow.smooth_vectors(vessel_position,vectors[keep])

In [ ]:
reg = morphotrack.flow.polynomial_fitting(vessel_position, smoothed_vectors, degree=5)

In [ ]:
# visualize if needed
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')


pred = normalize(morphotrack.flow.wrap_prediction(reg, vessel_position), axis=1)
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = pred

for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='prediction', colormap=c, blending='additive')

In [ ]:
pickle.dump(reg, open(os.path.join(io_directory,'model_.pkl'), 'wb'))